# Bibliotecas

In [30]:
import sys
from abc import ABC, abstractmethod
from collections import defaultdict

# Interface do Grafo

In [31]:
class Grafo(ABC):
    @abstractmethod
    def numero_de_vertices(self):
        pass

    @abstractmethod
    def numero_de_arestas(self):
        pass

    @abstractmethod
    def sequencia_de_graus(self):
        pass

    @abstractmethod
    def adicionar_aresta(self, u, v):
        pass

    @abstractmethod
    def remover_aresta(self, u, v):
        pass

    @abstractmethod
    def imprimir(self):
        pass
    
    #adicionando novos métodos

    @abstractmethod
    def is_simples(self):
        pass

    @abstractmethod
    def is_nulo(self):
        pass

    @abstractmethod
    def is_completo(self):
        pass



# Classe GrafoDenso usando matriz de adjacência 

In [40]:
class GrafoDenso(Grafo):
    def __init__(self, vertices):
       
        if isinstance(vertices, int):
            self.rotulos = list(range(vertices))
        else:
            self.rotulos = list(vertices)

        n = len(self.rotulos)
        self.matriz = [[0] * n for _ in range(n)]

    def numero_de_vertices(self):
        return len(self.rotulos)

    def numero_de_arestas(self):
        count = 0
        n = len(self.matriz)
        for i in range(n):
            for j in range(i + 1, n):  
                if self.matriz[i][j] == 1:
                    count += 1
        return count

    def sequencia_de_graus(self):
        graus = []
        n = len(self.matriz)
        for i in range(n):
            grau = sum(self.matriz[i])
            graus.append(grau)
        return graus

    def adicionar_aresta(self, u, v):
        i = self.rotulos.index(u)
        j = self.rotulos.index(v)
        self.matriz[i][j] = 1
        self.matriz[j][i] = 1

    def remover_aresta(self, u, v):
        i = self.rotulos.index(u)
        j = self.rotulos.index(v)
        self.matriz[i][j] = 0
        self.matriz[j][i] = 0

    def imprimir(self):
        print("Matriz de Adjacência:")
        print("   " + "  ".join(self.rotulos))
        for i, rot in enumerate(self.rotulos):
            print(rot, self.matriz[i])
#novos métodos
    def is_simples(self):
        n = self.numero_de_vertices()
        for i in range(n):
            #verificar se possui laços, a diagonal principal deve ser 0
            if self.matriz[i][i]!= 0:
                return False
            #verificar arestas múltiplas
            for j in range(i,n):
                if self.matriz[i][j] > 1:
                    return False
        return True
    
    def is_nulo(self):
        # grafo nulo = não tem aresta --> todas as células da matriz são 0
        n = self.numero_de_vertices()
        for i in range(n):
            for j in range(n):
                if self.matriz[i][j] != 0:
                    return False
        return True
    
    def is_completo(self):
        n = self.numero_de_vertices()
        # 1. para ser completo tem que ser simples
        if not self.is_simples():
            return False
        #2. é considerado completo se houver 0 ou 1 vértice
        if n <= 1:
            return True
        #3. Para ser completo todos os elementos fora a  diagonal principal devem ser 1
        for i in range(n):
            for j in range(n):
                if i != j and self.matriz[i][j] != 1:
                    return False
        return True
        

# Criando a classe GrafoEsparso usando lista de adjacências

In [ ]:
class GrafoEsparso(Grafo):
    def __init__(self, vertices):
        if isinstance(vertices, int):
            self.rotulos = [str(i) for i in range(vertices)]
        else:
            self.rotulos = list(vertices)

        #lista de adjacência: {vertice : [vizinhos]}
        self.lista = defaultdict(list)
    def numero_de_vertices(self):
        return len(self.rotulos)
    
    def numero_de_arestas(self):
        count = 0
        for v in self.lista:
            count += len(self.lista[v])
        return count //2 #pois cada aresta aparece duas vezes
    
    def sequencia_de_graus(self):
        graus = []
        for v in self.rotulos:
            grau = len(self.lista[v])
            graus.append((v,grau))
        return graus
    def adicionar_aresta(self, u, v):
        if u in self.rotulos and v in self.rotulos:
            self.lista[u].append(v)
            self.lista[v].append(u) # grafo não direcionado
    
    def remover_aresta(self, u, v):
        if u in self.rotulos and v in self.rotulos:
            if v in self.lista[u]:
                self.lista[u].remove(v) # remove somente uma ocorrência
            if u in self.lista[v]:
                self.lista[v].remove(u) # remove somente uma ocorrência

    def imprimir(self):
        print("lista de adjacencia:")
        for v in self.rotulos:
            print(f"{v}: {self.lista[v]}")

 #novos métodos

    def is_simples(self):
        #Um grafo simples não tem laços nem arestas múltiplas
        for v in self.lista:
            #verificar se tem laço
            if v in self.lista[v]:
                return False
            #verificar se tem arestas múltiplas
            if len(self.lista[v]) != len(set(self.lista[v])):
                return False
        return True
    
    def is_nulo(self):
        # Um grafo é nulo se não possui arestas
        return self.numero_de_arestas() == 0
    
    def is_completo(self):
        # Um grafo completo é um grafo simples onde cada vértice está conectado a todos os outros
        # 1. checar se é simples:
        if not self.is_simples():
            return False
        
        n = self.numero_de_vertices()
        #2. se o número de vértices for 0 ou 1 (considerando K0 completo), o grafo é completo
        if n <= 1:
            return True
        
        #o número de arestas em um grafo completo com n vértices é n * (n-1) /2
        arestas_completas = n * (n-1) // 2

        #verificar se o número de arestas bate com o número de arestas esperado
        if self.numero_de_arestas() == arestas_completas:
            return True
        else:
            return False
    
   

# Teste Grafo Denso

In [41]:


g = GrafoDenso(["A", "B", "C", "D", "E"])  # cria grafo 

 # Adiciona arestas {(A,B), (A,C), (C,D), (C,E), (B,D)}
g.adicionar_aresta("A", "B")
g.adicionar_aresta("A", "C")
g.adicionar_aresta("C", "D")
g.adicionar_aresta("C", "E")
g.adicionar_aresta("B", "D")

    # Imprime informações
print("\n--- Grafo Inicial ---")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())

 # Remove a aresta (A,C)
g.remover_aresta("A", "C")

 # Imprime novamente
print("\n--- Após remover a aresta (A,C) ---")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())







--- Grafo Inicial ---
Matriz de Adjacência:
   A  B  C  D  E
A [0, 1, 1, 0, 0]
B [1, 0, 0, 1, 0]
C [1, 0, 0, 1, 1]
D [0, 1, 1, 0, 0]
E [0, 0, 1, 0, 0]
Número de vértices: 5
Número de arestas: 5
Sequência de graus: [2, 2, 3, 2, 1]
O grafo é simples? True
O grafo é nulo? False
O grafo é completo? False

--- Após remover a aresta (A,C) ---
Matriz de Adjacência:
   A  B  C  D  E
A [0, 1, 0, 0, 0]
B [1, 0, 0, 1, 0]
C [0, 0, 0, 1, 1]
D [0, 1, 1, 0, 0]
E [0, 0, 1, 0, 0]
Número de vértices: 5
Número de arestas: 4
Sequência de graus: [1, 2, 2, 2, 1]
O grafo é simples? True
O grafo é nulo? False
O grafo é completo? False


# Teste Grafo Esparso

In [44]:
# Instância com rótulos {A,B,C,D,E}
g = GrafoEsparso(["A", "B", "C", "D", "E"])

# Adiciona arestas {(A,B), (A,C), (C,D), (C,E), (B,D)}
g.adicionar_aresta("A", "B")
g.adicionar_aresta("A", "C")
g.adicionar_aresta("C", "D")
g.adicionar_aresta("C", "E")
g.adicionar_aresta("B", "D")

# Adiciona aresta duplicada (A,B) para testar multigrafo
g.adicionar_aresta("A", "B")

print("\n--- Grafo Inicial ---")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())

# Remove só uma instância da aresta (A,B)
g.remover_aresta("A", "B")

print("\n-- Após remover uma aresta (A,B) --")
g.imprimir()
print("Número de vértices:", g.numero_de_vertices())
print("Número de arestas:", g.numero_de_arestas())
print("Sequência de graus:", g.sequencia_de_graus())
print("O grafo é simples?", g.is_simples())
print("O grafo é nulo?", g.is_nulo())
print("O grafo é completo?", g.is_completo())



--- Grafo Inicial ---
lista de adjacencia:
A: ['B', 'C', 'B']
B: ['A', 'D', 'A']
C: ['A', 'D', 'E']
D: ['C', 'B']
E: ['C']
Número de vértices: 5
Número de arestas: 6
Sequência de graus: [('A', 3), ('B', 3), ('C', 3), ('D', 2), ('E', 1)]
O grafo é simples? False
O grafo é nulo? False
O grafo é completo? False

-- Após remover uma aresta (A,B) --
lista de adjacencia:
A: ['C', 'B']
B: ['D', 'A']
C: ['A', 'D', 'E']
D: ['C', 'B']
E: ['C']
Número de vértices: 5
Número de arestas: 5
Sequência de graus: [('A', 2), ('B', 2), ('C', 3), ('D', 2), ('E', 1)]
O grafo é simples? True
O grafo é nulo? False
O grafo é completo? False
